In [4]:
# le plus important

import pandas as pd
import re

print("="*60)
print("NORMALISATION DES TRAITEMENTS VIH – ABRÉVIATIONS")
print("="*60)

# ------------------------------------------------------------------
# 1. Charger les fichiers
# ------------------------------------------------------------------
df_arv = pd.read_excel("data/ARV.xlsx")
df_acronym = pd.read_excel("data/medicaments_acronym.xlsx")

print(f"→ {len(df_acronym)} abréviations dans le dictionnaire")
print(f"→ {len(df_arv)} lignes ARV à traiter")

# ------------------------------------------------------------------
# 2. Préparation du dictionnaire d'abréviations
# ------------------------------------------------------------------
df_acronym["Abbreviation"] = df_acronym["Abbreviation"].fillna("").astype(str).str.strip()
df_acronym["Definition"] = df_acronym["Definition"].fillna("").astype(str).str.strip()

acronym_dict = {
    row["Abbreviation"]: row["Definition"]
    for _, row in df_acronym.iterrows()
    if row["Abbreviation"] and row["Definition"]
}

# Trier par longueur décroissante
acronym_dict = dict(sorted(acronym_dict.items(), key=lambda x: len(x[0]), reverse=True))

print(f"✓ Dictionnaire chargé : {len(acronym_dict)} abréviations")

# ------------------------------------------------------------------
# 3. Liste des abréviations effectivement utilisées
# ------------------------------------------------------------------
abrevs_trouvees = set()

# ------------------------------------------------------------------
# 4. Fonction de remplacement
# ------------------------------------------------------------------

def remplacer_abreviations(traitement):

    if pd.isna(traitement) or not str(traitement).strip():
        return traitement

    traitement_str = str(traitement).strip()
    items = [t.strip() for t in traitement_str.split("+")]

    resultats = []

    for item in items:
        item_modifie = item

        for abbrev, definition in acronym_dict.items():
            pattern = r'\b' + re.escape(abbrev) + r'\b'

            if re.search(pattern, item_modifie, flags=re.IGNORECASE):

                # 🔥 enregistrer l’abréviation trouvée
                abrevs_trouvees.add(abbrev)

                item_modifie = re.sub(
                    pattern,
                    definition,
                    item_modifie,
                    flags=re.IGNORECASE
                )
                break

        resultats.append(item_modifie)

    return " + ".join(resultats)

# ------------------------------------------------------------------
# 5. Application sur le DataFrame
# ------------------------------------------------------------------
print("\nTraitement en cours...")
df_arv["ARV_normalise"] = df_arv["ARV"].apply(remplacer_abreviations)

nb_changes = (df_arv["ARV"].astype(str) != df_arv["ARV_normalise"].astype(str)).sum()
nb_total = len(df_arv)

print(f"✓ Normalisation terminée")
print(f"  → Total de lignes traitées : {nb_total}")
print(f"  → Lignes modifiées : {nb_changes}")
print(f"  → Lignes inchangées : {nb_total - nb_changes}")

# ------------------------------------------------------------------
# 6. Créer le DataFrame des abréviations trouvées
# ------------------------------------------------------------------
df_abrevs = pd.DataFrame({
    "Abbreviation": sorted(list(abrevs_trouvees))
})
df_abrevs["Definition"] = df_abrevs["Abbreviation"].map(acronym_dict)

print("\nAbréviations réellement trouvées dans les données :", len(df_abrevs))

# Sauvegarde
df_abrevs.to_excel("data/out/abrevs_utilisees.xlsx", index=False)
print("✓ Fichier créé : data/out/abrevs_utilisees.xlsx")

# ------------------------------------------------------------------
# 7. Sauvegarde du fichier ARV normalisé
# ------------------------------------------------------------------
# df_arv.to_excel("data/out/arv_abbr.xlsx", index=False)
# print("\n" + "="*60)
# print("Fichier créé : data/out/arv_abbr.xlsx")
# print("="*60)


NORMALISATION DES TRAITEMENTS VIH – ABRÉVIATIONS
→ 323 abréviations dans le dictionnaire
→ 19220 lignes ARV à traiter
✓ Dictionnaire chargé : 313 abréviations

Traitement en cours...
✓ Normalisation terminée
  → Total de lignes traitées : 19220
  → Lignes modifiées : 130
  → Lignes inchangées : 19090

Abréviations réellement trouvées dans les données : 6
✓ Fichier créé : data/out/abrevs_utilisees.xlsx


In [1]:
# le plus important

import pandas as pd
import re

print("="*60)
print("NORMALISATION DES TRAITEMENTS VIH – ABRÉVIATIONS")
print("="*60)

# ------------------------------------------------------------------
# 1. Charger les fichiers
# ------------------------------------------------------------------
df_arv = pd.read_excel("data/ARV.xlsx")
df_acronym = pd.read_excel("data/medicaments_acronym.xlsx")

print(f"→ {len(df_acronym)} abréviations dans le dictionnaire")
print(f"→ {len(df_arv)} lignes ARV à traiter")

# ------------------------------------------------------------------
# 2. Préparation du dictionnaire d'abréviations
# ------------------------------------------------------------------
df_acronym["Abbreviation"] = df_acronym["Abbreviation"].fillna("").astype(str).str.strip()
df_acronym["Definition"] = df_acronym["Definition"].fillna("").astype(str).str.strip()

# Créer un dictionnaire trié par longueur décroissante
# (pour traiter les abréviations longues en premier, ex: "ATV/c" avant "ATV")
acronym_dict = {}
for _, row in df_acronym.iterrows():
    abbrev = row["Abbreviation"].strip()
    definition = row["Definition"].strip()
    if abbrev and definition:
        acronym_dict[abbrev] = definition

# Trier par longueur décroissante
acronym_dict = dict(sorted(acronym_dict.items(), key=lambda x: len(x[0]), reverse=True))

print(f"✓ Dictionnaire chargé : {len(acronym_dict)} abréviations")

# ------------------------------------------------------------------
# 3. Fonction de remplacement des abréviations
# ------------------------------------------------------------------

def remplacer_abreviations(traitement):
    """
    Remplace les abréviations trouvées par leurs définitions.
    Garde le texte original pour ce qui n'est pas trouvé.
    """
    if pd.isna(traitement) or not str(traitement).strip():
        return traitement

    traitement_str = str(traitement).strip()

    # Séparer par "+" pour traiter chaque médicament individuellement
    items = [t.strip() for t in traitement_str.split("+")]

    resultats = []
    for item in items:
        if not item:
            continue

        item_original = item
        item_modifie = item

        # Essayer de remplacer chaque abréviation dans l'item
        for abbrev, definition in acronym_dict.items():
            # Chercher l'abréviation dans l'item (insensible à la casse)
            # Pattern: l'abréviation doit être un mot entier ou suivi d'un espace/chiffre
            pattern = r'\b' + re.escape(abbrev) + r'\b'

            if re.search(pattern, item_modifie, flags=re.IGNORECASE):
                # Remplacer en préservant la casse originale si possible
                item_modifie = re.sub(
                    pattern,
                    definition,
                    item_modifie,
                    flags=re.IGNORECASE
                )
                # On sort de la boucle après le premier remplacement réussi
                break

        resultats.append(item_modifie)

    return " + ".join(resultats)

# ------------------------------------------------------------------
# 4. Appliquer sur tout le DataFrame
# ------------------------------------------------------------------

print("\nTraitement en cours...")
df_arv["ARV_normalise"] = df_arv["ARV"].apply(remplacer_abreviations)

# Statistiques
nb_changes = (df_arv["ARV"].astype(str) != df_arv["ARV_normalise"].astype(str)).sum()
nb_total = len(df_arv)

print(f"✓ Normalisation terminée")
print(f"  → Total de lignes traitées : {nb_total}")
print(f"  → Lignes modifiées : {nb_changes}")
print(f"  → Lignes inchangées : {nb_total - nb_changes}")

# ------------------------------------------------------------------
# 5. Afficher quelques exemples
# ------------------------------------------------------------------
print("\n" + "="*60)
print("EXEMPLES DE TRANSFORMATION:")
print("="*60)

# Afficher les 15 premières lignes où il y a eu un changement
exemples = df_arv[df_arv["ARV"].astype(str) != df_arv["ARV_normalise"].astype(str)].head(15)
if len(exemples) > 0:
    for idx, row in exemples.iterrows():
        print(f"\nAvant : {row['ARV']}")
        print(f"Après : {row['ARV_normalise']}")
else:
    print("Aucune transformation effectuée.")

# Afficher quelques exemples spécifiques pour vérification
print("\n" + "="*60)
print("VÉRIFICATION DES CAS SPÉCIFIQUES:")
print("="*60)

# Chercher des exemples avec "ABT", "Videx", "Zerit"
test_cases = df_arv[df_arv["ARV"].astype(str).str.contains("ABT|Videx|Zerit", case=False, na=False)].head(5)
for idx, row in test_cases.iterrows():
    print(f"\nAvant : {row['ARV']}")
    print(f"Après : {row['ARV_normalise']}")

# ------------------------------------------------------------------
# 6. Sauvegarde
# ------------------------------------------------------------------
df_arv.to_excel("data/out/arv_abbr.xlsx", index=False)
print("\n" + "="*60)
print("Fichier créé : data/out/arv_abbr.xlsx")
print("="*60)

NORMALISATION DES TRAITEMENTS VIH – ABRÉVIATIONS
→ 323 abréviations dans le dictionnaire
→ 19220 lignes ARV à traiter
✓ Dictionnaire chargé : 313 abréviations

Traitement en cours...
✓ Normalisation terminée
  → Total de lignes traitées : 19220
  → Lignes modifiées : 130
  → Lignes inchangées : 19090

EXEMPLES DE TRANSFORMATION:

Avant : Cobicistat + Emtriva + Prezista + TAF
Après : Cobicistat + Emtriva + Prezista + tenofovir alafenamide

Avant : Celsentri + Dolutegravir  + Norvir + Prezista + Truvada
Après : Celsentri + Dolutegravir + Norvir + Prezista + Truvada

Avant : Emtriva + Névirapine (Viramune) + TAF
Après : Emtriva + Névirapine (Viramune) + tenofovir alafenamide

Avant : E/C/F TAF
Après : E/C/F tenofovir alafenamide

Avant : GSK1265744 LA (Cabotegravir) + TMC278 LA (Rilpivirine)
Après : GSK1265744 long-acting (Cabotegravir) + TMC278 long-acting (Rilpivirine)

Avant : Dolutegravir  + Truvada
Après : Dolutegravir + Truvada

Avant : DRV/COBI/TAF/FTC
Après : darunavir/COBI/TAF/FT

In [2]:
df_modifie = df_arv[df_arv["ARV"].astype(str) != df_arv["ARV_normalise"].astype(str)]


In [3]:
df_modifie 


,ARV,ARV_normalise
48,Cobicistat + Emtriva + Prezista + TAF,Cobicistat + Emtriva + Prezista + tenofovir al...
105,Celsentri + Dolutegravir + Norvir + Prezista ...,Celsentri + Dolutegravir + Norvir + Prezista +...
183,Emtriva + Névirapine (Viramune) + TAF,Emtriva + Névirapine (Viramune) + tenofovir al...
667,E/C/F TAF,E/C/F tenofovir alafenamide
894,GSK1265744 LA (Cabotegravir) + TMC278 LA (Rilp...,GSK1265744 long-acting (Cabotegravir) + TMC278...
...,...,...
18113,Dolutegravir + Fuzeon,Dolutegravir + Fuzeon
18263,Celsentri + Dolutegravir + Emtriva + Intelenc...,Celsentri + Dolutegravir + Emtriva + Intelence...
18634,GSK1265744 LA (Cabotegravir),GSK1265744 long-acting (Cabotegravir)
18694,Dolutegravir + Norvir + Reyataz,Dolutegravir + Norvir + Reyataz


In [3]:
import pandas as pd 
df = pd.read_excel("data/out/arv_abbr.xlsx")

In [5]:
df


,ARV,ARV_normalise
0,Combivir + Kalétra,Combivir + Kalétra
1,Isentress + Truvada,Isentress + Truvada
2,Isentress + Kivexa,Isentress + Kivexa
3,Triumeq,Triumeq
4,Genvoya,Genvoya
...,...,...
19215,Fuzeon + Trizivir + Viread + TMC 114,Fuzeon + Trizivir + Viread + TMC 114
19216,Norvir + Truvada + Ziagen + TMC 114 (Darunavir...,Norvir + Truvada + Ziagen + TMC 114 (Darunavir...
19217,Norvir + Prezista + Truvada + Ziagen + TMC 125...,Norvir + Prezista + Truvada + Ziagen + TMC 125...
19218,Norvir + Prezista + Truvada + Ziagen + Maravir...,Norvir + Prezista + Truvada + Ziagen + Maravir...


In [7]:
df = df.drop(columns=['ARV'])          # Supprime la colonne ARV
df = df.rename(columns={'ARV_normalise': 'ARV'})   # Renomme ARV_normalise → ARV


In [9]:
df

,ARV
0,Combivir + Kalétra
1,Isentress + Truvada
2,Isentress + Kivexa
3,Triumeq
4,Genvoya
...,...
19215,Fuzeon + Trizivir + Viread + TMC 114
19216,Norvir + Truvada + Ziagen + TMC 114 (Darunavir...
19217,Norvir + Prezista + Truvada + Ziagen + TMC 125...
19218,Norvir + Prezista + Truvada + Ziagen + Maravir...


In [11]:
df.to_excel("data/out/arv_abbr.xlsx", index=False)
